In [3]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook as tqdm
import pickle

In [8]:
headers = {
    'authority': 'zsurpubi.hu',
    'accept': '*/*',
    'sec-fetch-dest': 'empty',
    'x-requested-with': 'XMLHttpRequest',
    'request-id': '|NvdVJ.YQzbH',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36',
    'content-type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'origin': 'https://zsurpubi.hu',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-mode': 'cors',
    'referer': 'https://zsurpubi.hu/kereses/?keyword=&date_from=2019.+01.+01.&date_to=2019.+12.+31.&author=&category=&tags=',
    'accept-language': 'en-GB,en;q=0.9,hu-HU;q=0.8,hu;q=0.7,en-US;q=0.6'}

params = (
    ('keyword', ''),
    ('date_from', '2019. 01. 01.'),
    ('date_to', '2019. 12. 31.'),
    ('author', ''),
    ('category', ''),
    ('tags', ''),
)

data = {
  'search_more_articles': '1',
  'limit': '3000',
  'offset': '0'
}

response = requests.post('https://zsurpubi.hu/kereses/', headers=headers, params=params, data=data)

In [9]:
soup = BeautifulSoup(response.content, 'html.parser')

In [10]:
links = ["https://zsurpubi.hu/" + tag.find("a").get("href") for tag in soup.find_all("h2")]

In [12]:
len(links)

2929

In [17]:
zsurpubi_dict = {}
for url in tqdm(links):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        datetime = soup.find("time").get("datetime")
        related_urls = []
        paragraphs = soup.find_all("p")
        for paragraph in paragraphs:
            if "MTI" in paragraph.text or " mti " in paragraph.text or "Mti" in paragraph.text:
                related_urls.append("MTI")
            if paragraph.find_all("a") is not None:
                for link in paragraph.find_all("a"):
                    if "http" in link.get('href'):
                        related_urls.append(link.get('href'))
                    else:
                        related_urls.append("https://zsurpubi.hu/" +  link.get('href'))
        zsurpubi_dict[url] = {
        "related_links" : related_urls,
        "publish_date" : datetime
    }

In [18]:
with open('zsurpubi_dict.p', 'wb') as f:
    pickle.dump(zsurpubi_dict, f)